In [1]:
!pip install transformers datasets torch nltk


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# Load the dataset and limit it to 5000 samples for training
dataset = load_dataset("ccdv/pubmed-summarization", split="train[:5000]")
val_dataset = load_dataset("ccdv/pubmed-summarization", split="validation[:1000]")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/207M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/59.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/58.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [3]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch

# Load the Pegasus tokenizer and model
model_name = "google/pegasus-cnn_dailymail"  # Pegasus model fine-tuned for summarization
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Move the model to the GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

In [4]:
def tokenize_function(examples):
    inputs = examples['article']
    targets = examples['abstract']
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")  # Reduced max_length

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize the train and validation datasets
tokenized_train = dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./pegasus-pubmed-small",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Small batch size to fit into memory
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate a larger batch size
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,  # Mixed precision training
    dataloader_pin_memory=True  # Pin memory for speed/memory efficiency
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
import torch

# Clear the GPU cache
torch.cuda.empty_cache()

# Create the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,3.605000,3.025385
2,3.299700,2.955907
3,3.230700,2.940890


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/tran

TrainOutput(global_step=1875, training_loss=3.344744140625, metrics={'train_runtime': 2343.8145, 'train_samples_per_second': 6.4, 'train_steps_per_second': 0.8, 'total_flos': 1.083549155328e+16, 'train_loss': 3.344744140625, 'epoch': 3.0})

In [7]:
def generate_summary(batch):
    inputs = tokenizer(batch['article'], max_length=256, truncation=True, padding='max_length', return_tensors="pt")
    inputs = inputs.to(device)  # Move inputs to GPU

    # Generate the output summary
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)

    # Decode the generated text
    summary = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return summary


In [12]:
import evaluate
import nltk

# Load the metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

# Define a function to calculate GLEU using nltk
def compute_gleu(predictions, references):
    gleu_scores = []
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.split()
        ref_tokens = [ref.split()]  # GLEU expects a list of reference sentences
        gleu_score = nltk.translate.gleu_score.sentence_gleu(ref_tokens, pred_tokens)
        gleu_scores.append(gleu_score)
    return sum(gleu_scores) / len(gleu_scores)

# Define a function to calculate the metrics
def compute_metrics(dataset):
    all_predictions = []
    all_references = []

    for sample in dataset:
        # Generate summary for each article
        generated_summary = generate_summary(sample)
        reference_summary = sample['abstract']  # Reference summary

        all_predictions.append(generated_summary[0])  # Get the first item in the list
        all_references.append(reference_summary)

    # ROUGE
    rouge_result = rouge.compute(predictions=all_predictions, references=[ref for ref in all_references])
    print("ROUGE Scores:", rouge_result)

    # BLEU
    bleu_result = bleu.compute(predictions=all_predictions, references=[[ref] for ref in all_references])
    print("BLEU Score:", bleu_result)

    # GLEU
    gleu_score = compute_gleu(all_predictions, all_references)
    print("GLEU Score:", gleu_score)

# Run the evaluation on the validation set (or a subset if it's too large)
compute_metrics(val_dataset.select(range(100)))  # Example: using the first 100 samples for evaluation


ROUGE Scores: {'rouge1': 0.30842967087286227, 'rouge2': 0.10524081557438916, 'rougeL': 0.19188251741535187, 'rougeLsum': 0.25556543318423053}
BLEU Score: {'bleu': 0.05239585301536865, 'precisions': [0.498974358974359, 0.1733678756476684, 0.09968586387434555, 0.06888888888888889], 'brevity_penalty': 0.33561356945294046, 'length_ratio': 0.47805834763422406, 'translation_length': 9750, 'reference_length': 20395}
GLEU Score: 0.09508073090383676


In [15]:
input_text = """
it occurs in more than 50% of patients and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.1 anemia is defined as an inadequate circulating level of hemoglobin ( hb ) ( hb < 12 g / dl ) and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy . preliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia.24 anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life ( qol ) and performance status in patients with cancer . thus , to improve physical functioning , qol , and prognosis in patients with cancer , it would be reasonable to take a proactive approach in identifying populations who need treatment for cancer - associated anemia ( caa ) and provide timely management . blood transfusion is an effective way to replace depleted hb within a short period , but the effect is , unfortunately , temporary and can cause serious adverse risks and increased mortality . in randomized clinical trials in patients with caa , erythropoiesis - stimulating agents ( esas ) produced significant increases in hb level , decreased transfusion requirements , and improved qol.57 however , 30%50% of patients do not respond to such agents . in addition , the use of esas often causes concern about severe adverse reactions.6,8 in several studies , esas were found to shorten overall survival time , or time to tumor progression in patients whose hb level reached more than 12 g / dl . these studies included patients with different primary cancers , such as breast , lung , head and neck , cervix , and lymphomas.911 the lack of response to erythropoietin stimulation in patients with cancer is partly attributed to the functional iron deficiency state , in which the high rate of erythropoiesis exceeds the delivery of usable iron , despite adequate iron stores.12 absolute iron deficiency , in contrast , occurs when iron delivery is impaired because iron stores are depleted ( serum ferritin , < 100 ng / ml ; transferring saturation , < 20%).13 hepcidin , a peptide hormone produced by the liver , is up - regulated in chronic inflammatory states including cancer . hepcidin inhibits iron transport across cell membranes , thus decreasing the accessibility of stored iron and gastrointestinal absorption of dietary iron , leading to an increased frequency of iron - restricted erythropoiesis.1416 many randomized trials examined the role of intravenous ( iv ) iron in addition to esas in the treatment of anemia in patients with cancer . many of these studies showed improvement in esa response , time to maximal response , reduction in esa dose , and improvement in qol parameters ( when measured ) in favor of the combination over esas alone . the observed benefit was independent of baseline iron parameters.1721 one study found a 36% reduction in the number of patients transfused.21 this pilot study assessed the efficacy and feasibility of iv iron monotherapy in patients with cancer who have anemia and who are undergoing treatment with chemotherapy and/or radiation therapy without the use of esas . patients received the study treatment for 12 weeks followed by a 4-week follow - up period . eligible patients were at least 18 years old , about to start a cycle of chemotherapy and/or radiation therapy within 1 week of inclusion , and had a nonmyeloid malignancy , hb levels of 11.0 g / dl or less , a life expectancy of more than 24 weeks , and an eastern cooperative oncology group performance status of 02 . patients were also required to have a serum ferritin level of 100 ng / ml or higher or transferrin saturation ( tsat ) levels of 15% or higher and to have received no esas or iv iron therapy within 30 days and no oral iron therapy ( 27 mg / day or more ) within 7 days before enrollment . patients were excluded for leukoerythroblastic features on blood film , hemolysis , gastrointestinal bleeding , folate or vitamin b12 deficiency , elevated serum ferritin ( 900 ng / ml ) or transferrin saturation ( tsat ) ( 35% ) levels , pregnancy or lactation , liver dysfunction ( grade 2 or higher based on national cancer institute common toxicity criteria ) , renal dysfunction ( serum creatinine levels 2.0 mg / dl ) , active infection requiring systemic antibiotics , personal or family history of hemochromatosis , comorbidities precluding study participation , hypersensitivity to iv iron , red blood cell transfusion within the last 2 weeks , or any investigational agent within 30 days before enrollment . ...
"""


In [16]:
# Tokenize and prepare the input for the model
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
inputs = {key: value.to(device) for key, value in inputs.items()}


In [17]:
# Generate the summary
summary_ids = model.generate(
    inputs['input_ids'],
    max_length=200,
    min_length=50,
    num_beams=4,
    early_stopping=True
)

# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Summary:")
print(summary)


Summary:
background : cancer - associated anemia ( caa ) occurs in more than 50% of patients and may reach 90% in certain types of cancers, especially in patients undergoing chemotherapy and/or radiation therapy. the purpose of this study was to assess the efficacy and feasibility of intravenous ( iv ) iron in patients with caa undergoing chemotherapy and/or radiation therapy without the use of erythropoiesis - stimulating agents ( esas ).materials and methods :patients and methods : a total of 339 patients with caa undergoing chemotherapy and/or radiation therapy were enrolled in this study. patients received iv iron therapy for 12 weeks followed by a 4-week follow - up period. patients were excluded for leukoerythroblastic features on blood film, hemolysis, gastrointestinal bleeding, folate or vitamin b12 deficiency, and liver dysfunction ( grade 2 or higher based on national cancer institute common toxicity criteria )
